# VibeVoice Colab — T4 Quickstart (1.5B)
This page provides a quickstart guide to run VibeVoice on Colab with T4.

The T4 GPU can only support the 1.5B model due to memory limitations. Please note that T4 can only use SDPA instead of flash_attention_2, which may result in unstable and lower audio quality. For the best TTS experience, we recommend trying the 7B model on a more powerful GPU.

## Step 1: Use T4



Use T4 in Colab: go to Runtime → Change runtime type → Hardware accelerator: GPU → T4.

In [ ]:
import torch
print(torch.cuda.is_available())
!nvidia-smi

## Step 2: Env Install

In [ ]:
!git clone https://github.com/shamspias/VibeVoice.git

import os
os.chdir("./VibeVoice")

!apt update && apt install ffmpeg -y
!pip install -e .

## Step 3: Run VibeVoice

In [ ]:
!python demo/inference_from_file.py --list_voices

In [ ]:
# --- edit only these two lines ---
SPEAKER_NAME = "Frank_man Mary_woman_bgm"     # e.g., "Mamaru"
TEXT_BASENAME = "2p_short"  # e.g., "2p_short" (no path, no .txt)
# --------------------------------

# Fixed bits (same path & extension)
BASE_TXT_DIR = "demo/text_examples"
TXT_EXT = ".txt"
MODEL_PATH = "microsoft/VibeVoice-1.5B"

# Build the full txt path from the base name
txt_path = f"{BASE_TXT_DIR}/{TEXT_BASENAME}{TXT_EXT}"

# Show what will run (optional)
print("Speaker:", SPEAKER_NAME)
print("Text path:", txt_path)

# Run the script (quotes handle spaces safely)


In [ ]:
# First download checkpoint takes ~3 minutes
!python demo/inference_from_file.py --model_path "{MODEL_PATH}" --txt_path "{txt_path}" --speaker_names "{SPEAKER_NAME}"

In [ ]:
from IPython.display import HTML, display
from pathlib import Path
import base64, html, os

# assumes TEXT_BASENAME is set, e.g. TEXT_BASENAME = "2p_short"
out_path = Path(f"./outputs/{TEXT_BASENAME}_generated.wav")

if not out_path.exists():
    display(HTML(f"""
    <div style="font-family: ui-sans-serif, system-ui; color:#b91c1c; background:#fff1f2;
                border:1px solid #fecdd3; border-radius:12px; padding:12px 16px;">
      <strong>Missing file:</strong> {out_path.as_posix()}
    </div>"""))
else:
    # Read and base64-encode the file so the browser can download it directly
    data = out_path.read_bytes()
    b64 = base64.b64encode(data).decode("ascii")
    filename = f"{TEXT_BASENAME}_generated.wav"
    title = f"Generated Audio · {TEXT_BASENAME}"

    html_block = f"""
    <div style="
        font-family: ui-sans-serif, system-ui, -apple-system, Segoe UI, Roboto, Arial;
        line-height:1.5; margin: 12px 0; padding: 18px;
        border: 1px solid #e5e7eb; border-radius: 16px;
        box-shadow: 0 2px 12px rgba(0,0,0,.06);
        background: linear-gradient(180deg,#ffffff,#fafafa);
    ">
      <div style="font-size: 20px; font-weight: 700; margin-bottom: 6px;">
        {html.escape(title)}
      </div>
      <div style="font-size: 12px; color:#6b7280; margin-bottom: 12px;">
        Size: {len(data)/1024:.1f} KB
      </div>

      <audio controls src="data:audio/wav;base64,{b64}"
             style="width:100%; outline:none; border-radius:10px; margin-bottom:14px;"></audio>

      <a href="data:audio/wav;base64,{b64}" download="{html.escape(filename)}"
         style="
           display:inline-flex; align-items:center; gap:10px;
           text-decoration:none; font-weight:600; font-size:16px;
           padding:12px 18px; border-radius:9999px;
           border: 1px solid #d1d5db;
           background: linear-gradient(180deg,#ffffff,#f3f4f6);
           box-shadow: 0 1px 3px rgba(0,0,0,.08), inset 0 1px 0 #ffffff;
           cursor:pointer;
         "
         onmouseover="this.style.boxShadow='0 2px 8px rgba(0,0,0,.12), inset 0 1px 0 #ffffff'"
         onmouseout="this.style.boxShadow='0 1px 3px rgba(0,0,0,.08), inset 0 1px 0 #ffffff'">
        <span aria-hidden="true" style="font-size:18px; line-height:1">⬇️</span>
        <span>Download WAV</span>
      </a>

      <div style="font-size:12px; color:#6b7280; margin-top:8px;">
        Tip: If your browser blocks data-URL downloads, right-click → “Save link as…”
      </div>
    </div>
    """
    display(HTML(html_block))


### TTS from your text

In [ ]:
text = """Speaker 1: Can I try VibeVoice with my own example?
Speaker 2: Of course! VibeVoice is open-source, built to benefit everyone — you’re welcome to try it out."""
with open("demo/text_examples/my_example.txt", "w", encoding="utf-8") as f:
    f.write(text)

In [ ]:
!python demo/inference_from_file.py --model_path microsoft/VibeVoice-1.5B --txt_path demo/text_examples/my_example.txt --speaker_names Alice Frank
Audio("./outputs/my_example_generated.wav")


# Risk and Limitations

While efforts have been made to optimize it through various techniques, it may still produce outputs that are unexpected, biased, or inaccurate. VibeVoice inherits any biases, errors, or omissions produced by its base model (specifically, Qwen2.5 1.5b in this release). Potential for Deepfakes and Disinformation: High-quality synthetic speech can be misused to create convincing fake audio content for impersonation, fraud, or spreading disinformation. Users must ensure transcripts are reliable, check content accuracy, and avoid using generated content in misleading ways. Users are expected to use the generated content and to deploy the models in a lawful manner, in full compliance with all applicable laws and regulations in the relevant jurisdictions. It is best practice to disclose the use of AI when sharing AI-generated content.